# Two Sigma News Baseline Test

TO DO:<br>
Do I use rows universe==0 ?  there are no PrevMktres values for these, but there is a NextMkres10 (so that can be put in PrevMkres10) !<br>
Check news_sample is not losing news items from news_train_df<br>


This Kernel goes through the bare minimum to produce a result. The sections are as follows.

* **Initialisation** Import and read the data. Define generic functions.<br>
* **Training Data - Market** Choose Market columns plus example<br>
* **Training Data - News** Choose News columns plus example<br>
* **Training Data - Merging Market with News** Merge procedure and test<br>
* **Training Data - Preparation** Sort by date and asset. Split into train and validation data<br>
* **Neural Net** Use LightGBM and do some checks of train and validation data<br>
* **Test data** Iterate through the test data and make predictions<br>
* **Submission** Write the submission file and instructions on how to use it as the competition entry<br>
* **Final word regarding the need to restart the kernel**<br>

#### Results (not great)
* Predictions using model: an RMSE score of: `0.06457`
* Predictionsusing model: a Kaggle Leaderboard score of: `-0.16318`
* All zero predictions: an RMSE score of: `0.06466` (only a tiny bit worse than the predictions)
* All zero predictions: a Kaggle Leaderboard score of: `0.0`
* Random predictions: an RMSE score of: `0.5807` (way worse then the predictions)
* Random predictions: a Kaggle Leaderboard score of: `-0.05323`

#### The Data<br>
<p align="left"> Data | <p align="left"> Dataframe|<p align="left"> Period
---|:---|:---
<p align="left"> market train data | <p align="left"> (4072956, 16) | <p align="left"> 1/2/2007 - 30/12/2016
<p align="left"> news train data | <p align="left"> (9328750, 35)**&ast;** | <p align="left"> 1/1/2007 - 30/12/2016
<p align="left"> market test data | <p align="left"> (&lt;assets per day>, 14)**&ast;&ast;** | <p align="left"> 3/1/2017 - 15/7/2019
<p align="left"> news test data | <p align="left"> (&lt;news items per day>, 35)**&ast;** | <p align="left"> 30/12/2016 - 14/7/2019

**&ast; news is rated for relevance, good, bad etc. how reliable is this?**<br>
**&ast;&ast; test data is given day by day, also without fields `universe` and `returnsOpenNextMktres10`**

#### Notes<br>
**universe==0** there are a lot of these and although there is no PrevMkres data the rest is there, including NextMkres10<br>
**Market returns** = ((close price – open price) / open price)   (ie. raw+unadjusted; eg. 120-100/100 = 0.2 = 20% return)<br>
**Market residualised returns** = market returns - benchmark returns; plus taking splits/dividents into account<br>
**Benchmark Market returns** = *how do I get this?*<br>
**`meanReturnsMkres1` and `meanReturnsMkres10`** would these new values be useful?<br>
**`returnsOpenPrevRaw10 - returnsOpenPrevMkres10`** will be benchmark return rate plus split/dividend calculation<br>
**`returnsOpenLastMktres10` and `returnsOpenNextMktres10`** should be the same 10 trading days apart<br>
**Output is `assetCode,confidenceValue`** (ie. predicting: `returnsOpenNextMktres10`); range [-1,1], 0 = least confidence<br>
**News data** *is it worth defining an algorithm to consolidate multiple news items for one stock on one day?*<br>
**`firstMentionSentence`** needs to be 0, 1 or 2 in order to trust the `sentimentNegative/Neutral/Positive` probabilities

#### Timeline<br>
25/09/2018: start<br>
02/01/2019:	team merger deadline<br>
08/01/2019:	submission deadline (select two best submissions to be rescored)<br>
15/07/2019:	end


#### End to end usage example from Two Sigma<br>
The training data can be read in one go (4 million and 9 million lines for market and news data respectively).<br>
You can only read in a day's worth of test data: ie. read, predict, read, predict, etc. 639 times for the 639 days of test data.<br>
*NOTE: create the functions `train_my_model` and `make_my_predictions` and uncomment all the three commented lines.*
```python
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
(market_train_df, news_train_df) = env.get_training_data()
#train_my_model(market_train_df, news_train_df)
i = 0
for (market_test_df, news_test_df, predictions_template_df) in env.get_prediction_days():
    predictions_df = predictions_template_df
    #predictions_df = make_my_predictions(market_test_df, news_test_df, predictions_template_df)
    env.predict(predictions_df)
    i = i + 1
print (i)
#env.write_submission_file()
```

# Initialisation
* Usual imports
* Set up the training data frames (save memory if you don't want submit test predictions)
* Define generic functions

In [4]:
import os
import sys
import re
import gc
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import lightgbm as lgb
from itertools import chain
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

py.init_notebook_mode(connected=True)
orig_max_rows = pd.options.display.max_rows
#pd.set_option('display.max_rows', 100000)
#pd.set_option('display.max_colwidth', 100)

submit_test = False  # keep / delete the env object (ie. training only)
use_news = True      # use / don't use news for training

market_train_df = pd.read_csv("../input/two-sigma-financial-news/marketdata.csv")
news_train_df   = pd.read_csv("../input/two-sigma-financial-news/newsdata.csv")

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:45: FutureWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [5]:
# This is only for running from Kaggle:
#from kaggle.competitions import twosigmanews
#env = twosigmanews.make_env()
#(market_train_df, news_train_df) = env.get_training_data()
#if submit_test is False:
#    print ("Not submitting test with this run so removing 'env' class to free memory")
#    del env
#    gc.collect()
# MEMORY CHECK: env uses 4.5 GB;  total: 5 GB (briefly hits 9GB though)
# MEMORY CHECK: dfs use: 2.8 GB;  total: 7.8 GB
# MEMORY CHECK: if submit_test is False: 5 GB

In [6]:
def summary(df, cols=None):
    if cols == None:
        cols = df.columns
    nrows = df.shape[0]
    if (nrows > orig_max_rows):
        pd.set_option('display.max_rows', nrows)
    print('DataFrame shape',df.shape)
    sdf = pd.DataFrame(index=cols, columns=['Null','dType','Type','Unique','Examples'])
    for col in cols:
        sys.stdout.write('.')
        sys.stdout.flush()
        sdf['Null'][col]     = df[col].isna().sum()
        sdf['Unique'][col]   = len(list(df[col].unique()))
        sdf['dType'][col]    = df[col].dtypes
        sdf['Type'][col]     = "-" if df[col].notna().sum() == 0 else type(df[col].dropna().iloc[0])
        sdf['Examples'][col] = "-" if df[col].notna().sum() == 0 else list(df[col].unique())
    return sdf.fillna('-')

# return (1) everything; (2) rows where 'time' == datefrom or (3) rows within a date range
def get_dates(df, datefrom=None, dateto=None):
    if datefrom is None:
        return (df['time'] != None)
    elif dateto is None:
        return (df['time'] == datefrom)
    else:
        return ((df['time'] >= datefrom) & (df['time'] <= dateto))

def df_memoryMB(df):
    return df.memory_usage().sum() / (1024 * 1024)

# Recursively finds size of objects
def get_class_size(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Mark as "seen" before entering recursion to cater for self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_class_size(v, seen) for v in obj.values()])
        size += sum([get_class_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_class_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_class_size(i, seen) for i in obj])
    return size

# get rough'n'ready memory calculation for class
def class_memoryMB(obj):
    return get_class_size(obj) / (1024 * 1024)

def show_year_rows_before_time_conversion(marketdf, newsdf):
    def show_year_rows(df, dfname):
        yearlist = df['time'].dt.year.tolist()
        print ("{:s} train rows for each year:".format(dfname))
        for year in sorted(set(yearlist)):
            print ("{:d}: {:d}".format(year, yearlist.count(year)))
    show_year_rows(marketdf, "market")
    show_year_rows(newsdf,   "news")

In [17]:
# converted time already
market_start_date = market_train_df['time'].head(1).values[0]
market_end_date = market_train_df['time'].tail(1).values[0]
news_start_date = news_train_df['time'].head(1).values[0]
news_end_date = news_train_df['time'].tail(1).values[0]
print ("market train dates:", market_start_date,"to",market_end_date)
print ("news   train dates:", news_start_date,  "to",news_end_date)
print ("market train shape:", market_train_df.shape)
print ("news   train shape:", news_train_df.shape)
print ("market train memory (MB):",df_memoryMB(market_train_df))
print ("news   train memory (MB):",df_memoryMB(news_train_df))
#show_year_rows_before_time_conversion(market_train_df, news_train_df)

market train dates: 20100104 to 20161230
news   train dates: 20100101 to 20161230
market train shape: (2946739, 17)
news   train shape: (7003400, 36)
market train memory (MB): 382.1912612915039
news   train memory (MB): 1876.7885971069336


# Training Data - Market
Show summary of `market_train_df`

Change 'time' to an integer YYYYMMDD. (Integer not string for flexability)<br>

Keep all universe entries for now.<br>

There are 93054 missing entries in `returnsOpenPrevMktres10` column so fill this in using `returnsOpenNextMktres10` and drop any rows that have missing cells (ie. NaN) in critical columns.<br>

*Note on the columns:*<br>
Residualised returns are surely the most useful columns since they show the real picture, ie. they take into consideration splits, dividends and overall market returns.<br>

*Note on Benchmark Returns:*<br>
`Raw - Mktres` doesn't equal benchmark returns because specific splits/dividends are taken into account, eg.<br>
`Date      : Asset : Days:  Raw      -  Mktres   = Benchmark + Split/Dividend Calc.`<br>
`2016-06-30: AAPL.O:    1:  0.005002 -  0.003236 = 0.001766`<br>
`2016-06-30: AAPL.O:   10: -0.020840 - -0.025209 = 0.004369`<br>
`2016-06-30: GOOG.O:    1:  0.003616 -  0.003339 = 0.00027`<br>
`2016-06-30: GOOG.O:   10: -0.041180 - -0.046054 = 0.004874`<br>

*Would the mean of mktres1 and the mean of mktres10 be more or less equal to the benchmark?*

In [18]:
summary(market_train_df)

DataFrame shape (2946739, 17)
.................

,Null,dType,Type,Unique,Examples
Unnamed: 0,0,int64,<class 'numpy.int64'>,2946739,"[1126217, 1126218, 1126219, 1126220, 1126221, ..."
time,0,int64,<class 'numpy.int64'>,1762,"[20100104, 20100105, 20100106, 20100107, 20100..."
assetCode,0,object,<class 'str'>,3327,"[A.N, AAI.N, AAP.N, AAPL.O, AAV.N, AAWW.O, AB...."
assetName,0,object,<class 'str'>,3139,"[Agilent Technologies Inc, AirTran Holdings In..."
volume,0,float64,<class 'numpy.float64'>,1945568,"[2729240.0, 3436803.0, 1701655.0, 17633150.0, ..."
close,0,float64,<class 'numpy.float64'>,51654,"[31.3, 5.18, 40.38, 214.01, 6.83, 38.2, 28.4, ..."
open,0,float64,<class 'numpy.float64'>,43177,"[31.39, 5.26, 40.7, 213.5, 6.61, 37.82, 28.51,..."
returnsClosePrevRaw1,0,float64,<class 'numpy.float64'>,2176952,"[0.007402639201802952, -0.007662835249042098, ..."
returnsOpenPrevRaw1,0,float64,<class 'numpy.float64'>,2155901,"[0.011275773195873029, 0.011538461538461496, -..."
returnsClosePrevMktres1,10668,float64,<class 'numpy.float64'>,2936037,"[-0.002343702903903977, -0.027851042066156708,..."


In [5]:
if market_train_df['time'].dtype != 'int':
    market_train_df['time'] = market_train_df['time'].dt.strftime(date_format='%Y%m%d').astype(int).values
    print ("Converted market time to integer YYYYMMDD (and removed time)")

# returnsOpenPrevMktres10's 93054 empty values can be filled using returnsOpenNextMktres10
# NOTE: asset data will leak into adjacent assets; so we'll remove the first and last 11 days, below. 
# ALSO: drop if any of volume, returnsOpenPrevMktres10 or returnsOpenNextMktres10 are NaN
#if (market_train_df['returnsOpenPrevMktres10'].isna().sum() > 0):
#    market_train_df.sort_values(by=['assetCode','time'], inplace=True)
#    market_train_df['returnsOpenPrevMktres10'] = market_train_df['returnsOpenNextMktres10'].shift(11)
#    market_train_df.sort_values(by=['time','assetCode'], inplace=True)
#    market_train_df.dropna(subset=['volume', 'returnsOpenPrevMktres10', 'returnsOpenNextMktres10'], how='any', inplace=True)
#    print ("Filled returnsOpenPrevMktres10 and dropped rows with NaN in critical columns")

if ('universe' in market_train_df.columns):
    universe0_count = market_train_df['universe'].loc[market_train_df['universe'] != 1.0].count()
    market_train_df = market_train_df.loc[market_train_df['universe'] == 1.0]
    market_train_df.drop(columns=['universe'], inplace=True)
    print ("Removed {:d} rows where universe != 1 and dropped column".format(universe0_count))

print(market_train_df.shape)
summary(market_train_df)

Converted market time to integer YYYYMMDD (and removed time)
(4072956, 16)
DataFrame shape (4072956, 16)
................

,Null,dType,Type,Unique,Examples
time,0,int64,<class 'numpy.int64'>,2498,"[20070201, 20070202, 20070205, 20070206, 20070..."
assetCode,0,object,<class 'str'>,3780,"[A.N, AAI.N, AAP.N, AAPL.O, ABB.N, ABC.N, ABD...."
assetName,0,category,<class 'str'>,3511,"[Agilent Technologies Inc, AirTran Holdings In..."
volume,0,float64,<class 'numpy.float64'>,2392152,"[2606900.0, 2051600.0, 1164800.0, 23747329.0, ..."
close,0,float64,<class 'numpy.float64'>,55434,"[32.19, 11.12, 37.51, 84.74, 18.02, 52.37, 23...."
open,0,float64,<class 'numpy.float64'>,46627,"[32.17, 11.08, 37.99, 86.23, 18.01, 52.4, 24.1..."
returnsClosePrevRaw1,0,float64,<class 'numpy.float64'>,2903234,"[0.005937500000000151, 0.004516711833784992, -..."
returnsOpenPrevRaw1,0,float64,<class 'numpy.float64'>,2866950,"[0.005312499999997611, -0.007168458781361964, ..."
returnsClosePrevMktres1,15980,float64,<class 'numpy.float64'>,4056918,"[nan, -0.019682215017981994, 0.010968392128086..."
returnsOpenPrevMktres1,15988,float64,<class 'numpy.float64'>,4056911,"[nan, -0.0018015662911102332, -0.0084749011130..."


**Set Market sample based on date range (chosen after adding whatever cols above)**

In [6]:
#trainfrom = 20100101
#trainto   = 20161230
#print ("Getting market samples from",trainfrom,"to",trainto)
#market_sample = market_train_df.loc[get_dates(market_train_df, datefrom=trainfrom, dateto=trainto)]
market_sample = market_train_df

Getting market samples from 20100101 to 20161230


#### Example six months look at Google (Alphabet) GOOG.O:
* a table shows `returnsOpenPrevMktres10` and `returnsOpenNextMktres10` are the same 10 trading days apart.
* the graph shows that GOOG.O in blue (no voting rights) and GOOGL.O in orange (voting rights) track one another

In [19]:
# For expressions you can set: flags=re.IGNORECASE, regex=True
def get_asset(df, cols, acol, asset, flags=0, regex=False, datefrom=None, dateto=None):
    search_asset = ((df[acol].str.contains(asset, flags=flags, regex=regex)) &
                    get_dates(df, datefrom=datefrom, dateto=dateto))
    return df[cols].loc[search_asset]

eg_from = 20160701
eg_to   = 20161230
eg1_df = get_asset(market_train_df, market_train_df.columns, 'assetCode', 'GOOG.O', datefrom=eg_from, dateto=eg_to)
eg2_df = get_asset(market_train_df, market_train_df.columns, 'assetCode', 'GOOGL.O', datefrom=eg_from, dateto=eg_to)
print ("6 months of GOOG.O",eg1_df.shape)
print ("6 months of GOOGL.O",eg2_df.shape)
print ("First 20 lines of GOOG.O")
eg1_df[['time','assetCode','assetName','returnsOpenPrevMktres10','returnsOpenNextMktres10']].head(20)

6 months of GOOG.O (127, 17)
6 months of GOOGL.O (127, 17)
First 20 lines of GOOG.O


,time,assetCode,assetName,returnsOpenPrevMktres10,returnsOpenNextMktres10
2715726,20160701,GOOG.O,Alphabet Inc,-0.037190,0.000302
2717523,20160705,GOOG.O,Alphabet Inc,-0.016026,0.009272
2719322,20160706,GOOG.O,Alphabet Inc,-0.011616,0.007033
2721120,20160707,GOOG.O,Alphabet Inc,-0.007512,0.016169
2722917,20160708,GOOG.O,Alphabet Inc,-0.004890,0.015183
2724716,20160711,GOOG.O,Alphabet Inc,0.029450,0.007083
2726516,20160712,GOOG.O,Alphabet Inc,-0.010521,0.009625
2728316,20160713,GOOG.O,Alphabet Inc,-0.032960,0.030083
2730113,20160714,GOOG.O,Alphabet Inc,-0.019475,0.062852
2731910,20160715,GOOG.O,Alphabet Inc,-0.002983,0.048494


In [20]:
def goscatter(df, col):
    return go.Scatter(x=df['time'].values, y=df[col].values)

# trace 0: blue, trace 1: orange
#data = [goscatter(eg1_df, 'returnsOpenPrevMktres10'), goscatter(eg1_df, 'returnsOpenNextMktres10')]
data = [goscatter(eg1_df, 'returnsOpenPrevMktres10'), goscatter(eg2_df, 'returnsOpenPrevMktres10')]
layout = go.Layout(title="returnsOpenPrevMktres10: GOOG.O vs GOOGL.O",
                   xaxis=dict(title='Date', showticklabels=False), yaxis=dict(title='USD'))
py.iplot(go.Figure(data=data, layout=layout))

# Training Data - News
Show summary of `news_train_df`

Change 'time' to an integer YYYYMMDD. (Integer not string for flexability)<br>

*Note on columns:*
There can be several articles on the same day for a specific asset. Lumping them all together so that they can be treated as one is going to be tricky since an alert (`urgency == 1`) is probably more weighty than an article (`urgency == 3`).

In [ ]:
summary(news_train_df)

In [7]:
if use_news:
    if news_train_df['time'].dtype != 'int':
        news_train_df['time'] = news_train_df['time'].dt.strftime(date_format='%Y%m%d').astype(int).values
        print ("Converted news time to integer YYYYMMDD (and removed time)")

    print ("Getting news samples from",trainfrom,"to",trainto)
    news_sample = news_train_df.loc[get_dates(news_train_df, datefrom=trainfrom, dateto=trainto)]

Converted news time to integer YYYYMMDD (and removed time)
Getting news samples from 20100101 to 20161230


**Example finding news for GOOG.O**

In [ ]:
# For expressions you can set: flags=re.IGNORECASE, regex=True
def get_news(df, cols, acol, asset, flags=0, regex=False, datefrom=None, dateto=None):
    search_asset = ((df[acol].str.contains(asset, flags=flags, regex=regex)) &
                    get_dates(df, datefrom=datefrom, dateto=dateto))
    return df[cols].loc[search_asset]
    
#get_news(news_train_df, news_train_df.columns, 'assetCodes', 'GOOG.O', datefrom=20160630, dateto=20171230).head(6)
goog_one_day_eg = get_news(news_train_df, news_train_df.columns, 'assetCodes', 'GOOG.O', datefrom=20160630)
print (goog_one_day_eg.shape[0],"GOOG.O news items for one day example. Below are the first 3...")
goog_one_day_eg.head(3)

# Training Data - Merging Market with News
Define the functions we'll use to manipulate the market and news dataframes.

Define the dates we are happy to use.<br>
*NOTE: including 2007-2009 gives strange results when generating comparative "all zero precictions". I get an RMSE of 9.4 so I assume there are some spikes in the data that affect affect this average). Therefore don't use this period.*

Get market samples using whatever columns we think are most important.

Define the market and news columns which get the best result:<br>
* `asset_predictors`: those columns I want in the market data
* `select_news_cols` is used to select data initially from `news_train_df`
* `append_news_cols` is asset_predictors with keys (time and assetCode (singular)) for merging news and market data

Get news samples. News is expanded there is only one asset per row.

Split and consolidate news into 8 manageable and meaningful "news item" chunks for each day and asset. The chunks are:
* urgency = 1 and firstMentionSentence = 0
* urgency = 1 and firstMentionSentence = 1
* urgency = 1 and firstMentionSentence = 2
* urgency = 1 and firstMentionSentence = >=3
* urgency = 3 and firstMentionSentence = 0
* urgency = 3 and firstMentionSentence = 1
* urgency = 3 and firstMentionSentence = 2
* urgency = 3 and firstMentionSentence = >=3<br>
*NOTE: the sentiment values are added together - do I want to squash the resultant value (eg. with sigmoid)?*<br>
*NOTE: what do I do with marketCommentary?*

Then merge the news chunks with the market data. The Market sample will have 58 columns 10 + (8 x 6).

Display part of the market sample which has news for verification.

In [ ]:
# remove all non-(alphanum|dots|commas), then split using the comma and flatten list-of-lists
def one_asset_per_row(ser):
    return list(chain.from_iterable(ser.str.replace('[^\w\.\,]','').str.split(',')))

def get_asset_count_arr(df):
    return df['assetCodes'].str.split(',').map(len)

def expand_assetCodes(df, cols):
    assetCodes = one_asset_per_row(df['assetCodes'])  # list of assets
    asset_count_arr = get_asset_count_arr(df)         # how above assets are grouped
    newdf = pd.DataFrame(columns=cols)
    for col in cols:
        if (col == 'assetCode'):
            newdf['assetCode'] = assetCodes
        else:
            newdf[col] = np.repeat(df[col].values, asset_count_arr)
    return newdf

def get_news_search(df, urgency, firstMentionSentence):
    if (firstMentionSentence == 'N'):
        return ((df['urgency'] == urgency) &
                (df['firstMentionSentence'] >= 3))
    else:
        return ((df['urgency'] == urgency) &
                (df['firstMentionSentence'] == firstMentionSentence))

def get_news_sample(df, search, keys):
    return df.loc[search].groupby(keys, as_index=False, sort=False).sum(numeric_only=True)

def merge_mn(marketdf, newsdf, keys, how, suffixes):
    return pd.merge(marketdf, newsdf, on=keys, how=how, suffixes=suffixes)

# Append relevant newsdf to marketsdf
def OLD_split_merge_news(marketdf, newsdf, keys):
    sum_newsdf = newsdf.groupby(keys, as_index=False, sort=False).sum(numeric_only=True)
    return merge_mn(marketdf, sum_newsdf, keys, 'left', ['',''])

# Append relevant newsdf to marketsdf
def split_merge_news(marketdf, newsdf, keys):
    search_alert0 = get_news_search(newsdf, 1, 0)
    search_alert1 = get_news_search(newsdf, 1, 1)
    search_alert2 = get_news_search(newsdf, 1, 2)
    search_alertN = get_news_search(newsdf, 1, 'N')
    search_article0 = get_news_search(newsdf, 3, 0)
    search_article1 = get_news_search(newsdf, 3, 0)
    search_article2 = get_news_search(newsdf, 3, 0)
    search_articleN = get_news_search(newsdf, 3, 'N')
    alert0df = get_news_sample(newsdf, search_alert0, keys)
    alert1df = get_news_sample(newsdf, search_alert1, keys)
    alert2df = get_news_sample(newsdf, search_alert2, keys)
    alertNdf = get_news_sample(newsdf, search_alertN, keys)
    article0df = get_news_sample(newsdf, search_article0, keys)
    article1df = get_news_sample(newsdf, search_article1, keys)
    article2df = get_news_sample(newsdf, search_article2, keys)
    articleNdf = get_news_sample(newsdf, search_articleN, keys)
    marketdf = merge_mn(marketdf, alert0df, keys, 'left', ['',''])
    marketdf = merge_mn(marketdf, alert1df, keys, 'left', ['_alert0','_alert1'])
    marketdf = merge_mn(marketdf, alert2df, keys, 'left', ['',''])
    marketdf = merge_mn(marketdf, alertNdf, keys, 'left', ['_alert2','_alertN'])
    marketdf = merge_mn(marketdf, article0df, keys, 'left', ['',''])
    marketdf = merge_mn(marketdf, article1df, keys, 'left', ['_article0','_article1'])
    marketdf = merge_mn(marketdf, article2df, keys, 'left', ['',''])
    marketdf = merge_mn(marketdf, articleNdf, keys, 'left', ['_article2','_articleN'])
    return marketdf

**For re-runs start here**

In [ ]:
# for re-runs since memory is a bit tight
try:
    sys.getsizeof(trainx)
    del market_sample
    del news_sample
    del trainx
    del validx
    gc.collect()
    print ("Memory freed")
except:
    print ("First run - no memory can be freed")

In [ ]:
# MARKET: drop: 'time', 'assetCode', 'assetName', 'returnsOpenNextMktres10'
market_cols = ['time', 'assetCode', 'assetName', 'volume',
               'returnsOpenPrevMktres1', 'returnsOpenPrevMktres10',
               'returnsClosePrevMktres1', 'returnsClosePrevMktres10',
               'returnsOpenNextMktres10']
# MARKET: -or- not drop, that is the question
#market_cols = market_train_df.columns
print ("market_cols:", market_cols)
print ("")
market_sample = market_sample[market_cols]
print ("Market train shape is ", market_train_df.shape)
print ("Market sample shape is", market_sample.shape)

if use_news:
    # NEWS: asset_predictors are the news columns I want to in market data
    # NEWS: select_news_cols is to select the information initially from news_train_df
    # NEWS: append_news_cols is asset_predictors with keys (time and assetCode (singular)) for merging news with market data
    #asset_predictors = ['urgency', 'marketCommentary', 'firstMentionSentence',
    #                    'sentimentNegative', 'sentimentNeutral', 'sentimentPositive']
    # NEWS: NOT APPENDED: 'time', 'headline', assetCode'
    # NEWS: NOT APPENDED: 'time', 'sourceTimestamp', 'firstCreated', 'sourceId', 'assetName',
    # NEWS: NOT APPENDED: 'takeSequence', 'provider', 'subjects', 'audiences', 'headlineTag',
    asset_predictors = ['urgency', 'marketCommentary',
       'sentenceCount', 'wordCount', 'firstMentionSentence', 'relevance', 'sentimentClass',
       'sentimentNegative', 'sentimentNeutral', 'sentimentPositive']
    #asset_predictors = ['urgency', 'bodySize', 'companyCount', 'marketCommentary',
    #   'sentenceCount', 'wordCount', 'firstMentionSentence', 'relevance', 'sentimentClass',
    #   'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
    #   'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
    #   'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
    #   'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D']
    select_news_cols = ['time', 'headline', 'assetCodes'] + asset_predictors
    append_news_cols = ['time', 'assetCode'] + asset_predictors   # NOTE: 'assetCode' will exist when this is used
    print ("select_news_cols:", select_news_cols)
    print ("")  
    print ("Expanding news:")
    news_sample = news_sample[select_news_cols]
    print ("News train shape is   ", news_train_df.shape)
    print ("News sample shape is  ", news_sample.shape)
    news_sample = expand_assetCodes(news_sample, append_news_cols)
    print ("News expanded shape is", news_sample.shape)

    print ("Merging news and market data")
    market_sample = split_merge_news(market_sample, news_sample, ['time','assetCode'])

    print ("Market plus News sample shape is", market_sample.shape)
    print ("Market plus News sample - first 5 entries with at least one urgency>0 news item")
    market_sample.loc[market_sample['urgency_alert0'] > 0].head(5).T
    #market_sample.loc[market_sample['urgency'] > 0].head(5).T

In [ ]:
summary(market_sample)

# Training Data - Preparation
Drop unused columns.

Opportunity to sort the values, however, it's already sorted by time and assetCode and this seems to work better than assetCode then time.<br>
Split into training and validation data.

Save labels time, assetCode and assetName for checks later on.

Encode the assetCode.

Any strings dropped.

Some checks.

In [ ]:
def get_unused(df):
    rows = df.shape[0]
    ddf = df.isna().sum()
    return list(ddf[ddf >= rows].index)

def encode_col(df, col, uvalues):
    le = LabelEncoder()
    le.fit(uvalues)
    return le.transform(df[col].astype('str'))

def split_data(df, sortkeys=None, test_size=0.1, shuffle=True):
    tx, vx, ty, vy = train_test_split(df, df['returnsOpenNextMktres10'], test_size=test_size, shuffle=shuffle)
    ty = ty.fillna(0).values
    vy = vy.fillna(0).values
    if (sortkeys!=None):
        tx = tx.sort_values(by=sortkeys)
        vx = vx.sort_values(by=sortkeys)
    return tx, vx, ty, vy

def convert_to_float(df):
    for col in df.columns:
        df[col] = df[col].astype(float)
    return df

In [ ]:
# Drop unused
train_unused_cols = get_unused(market_sample)
market_sample.drop(columns=train_unused_cols, inplace=True)
print ("Dropped these unused columns from market data:", train_unused_cols)
print ("")
rows_before = market_sample.shape[0]
market_sample.dropna(how='any', inplace=True)
rows_after = market_sample.shape[0]
print ("Dropped {:d} rows from market data due to NaN value".format(rows_before-rows_after))
print ("")

print ("Splitting the data into trainx/y and validx/y")
#sortkeys = ['time','assetCode']
#sortkeys = ['assetCode', 'time']
sortkeys = None
trainx, validx, trainy, validy = split_data(market_sample.copy(), sortkeys=sortkeys, test_size=0.1, shuffle=False)
_time = validx['time'].values   # HACK for one of the NeuralNets trying to use lgb.get_field() 

print ("Saving labels for checking: time, assetCode and assetName")
trainx_label = trainx[['time', 'assetCode', 'assetName']]
validx_label = validx[['time', 'assetCode', 'assetName']]

print ("No column needs encoding")
#print ("Encode assetCode")
#unique_assetCodes = market_sample['assetCode'].astype(str).unique()
#trainx['assetCode'] = encode_col(trainx, 'assetCode', unique_assetCodes)
#validx['assetCode'] = encode_col(validx, 'assetCode', unique_assetCodes)

# set "string" columns to remove based on encoding (above)
train_str_cols = ['time', 'assetCode', 'assetName']
#train_str_cols = ['assetName']
print ("Dropped these columns from trainx:", train_str_cols + ['returnsOpenNextMktres10'])
trainx.drop(columns=train_str_cols + ['returnsOpenNextMktres10'], inplace=True)
validx.drop(columns=train_str_cols + ['returnsOpenNextMktres10'], inplace=True)

print ("Converting everything to float")
trainx = convert_to_float(trainx)
validx = convert_to_float(validx)

# Checks
assert set(trainx.columns) == set(validx.columns)
print ("trainx shape:", trainx.shape, "trainy (numpy) shape:", trainy.shape)
print ("validx shape:", validx.shape, "validy (numpy) shape:", validy.shape)
summary(validx)

# Neural Net
So far, the predictions are not much better than using all zeros for predictions. Definitely room for improvement.<br>
When I get better results I can work on converting the y values into confidence values between -1 and 1.

Notes:<br>
Score is calculated as follows:<br>
1. x_timearray = Sum (sigma) of one day's assets (confidence * Mktres10 * universe)<br>
2. score = mean(x_timearray) / stddev(x_timearray)
*NOTE on universe in training:<br>
Including the universe does not make sense since it is not in the test data. The more universe==0 assets there are in a day's trading the larger the affect will be on the means and standard deviations. Is this not training for something that will never appear in test?*

Tried:<br>
* Changing the date: I get slightly better results with 1/1/2010 - 30/12/2016 (ie. ignore 2007-2009)
* Encoding time and assetCode: not much difference - left them out
* More market cols: not much difference - kept only `volume` and `returnsOpenPrevMktres10`
* More news cols: 

Calculations are:<br>
* RMSE trainy - predictions of trainy (to see how "trained" the model is; with no consideration for overfitting)
* RMSE trainy - predictions of all zero
* RMSE trainy - predictions with are random
* **RMSE validy - predictions of validy (the best indicator a good model for this kernel)**
* RMSE validy - predictions of all zero
* RMSE validy - predictions which are random
* all the above repeated using MAE (Mean of Absolute Error) for comparison

Prediction scores for: **validy - predictions of validy**
* RMSE: `0.5812` USING: all random
* RMSE: `0.0658` USING: all zeros
* RMSE: `0.0626` USING: sort:time,asset; date:2010-end; market cols: bare minimum

*NOTE: sort:time,asset and sort:None are the same*

In [ ]:
def train_nn(trainx, trainy, validx, validy):
    params = {
        "objective"               : "regression_l1",
        "metric"                  : "None", 
        "num_leaves"              : 15,
        "learning_rate"           : 0.1,
        "max_depth"               : -1,
        "min_data_in_leaf"        : 20,
        "min_sum_hessian_in_leaf" : 1e-3,
        "bagging_fraction"        : 0.5,
        "feature_fraction"        : 1.0,
        "bagging_freq"            : 2,
        "lambda_l1"               : 0,
        "lambda_l2"               : 0
    }
    def score(_predvy, _validlgb):
        _validy = _validlgb.get_label()
        #x = pd.DataFrame({'sigma': (_predvy * _validy * _universe)}).groupby(_time).sum()
        x = pd.DataFrame({'sigma': (_predvy * _validy)}).groupby(_time).sum()
        return 'sigma_score', (x['sigma'].mean() / x['sigma'].std()), True

    evals_result = {}
    #_time     = validx['time'].values      # because _validlgb.get_field() does not work
    #_universe = validx['universe'].values  # because _validlgb.get_field() does not work
    trainlgb = lgb.Dataset(trainx, label=trainy)
    validlgb = lgb.Dataset(validx, label=validy)
    return lgb.train(params, trainlgb, num_boost_round=500, valid_sets=(validlgb), valid_names=('valid'), verbose_eval=25,
               early_stopping_rounds=25, feval=score, evals_result=evals_result)
    #df_result = pd.DataFrame(evals_result['valid'])

nn = train_nn(trainx, trainy, validx, validy)
predvy = nn.predict(validx, num_iteration=nn.best_iteration)
predty = nn.predict(trainx, num_iteration=nn.best_iteration)

lgb.plot_importance(nn, figsize=(10,10), height=0.8, grid=False);

In [ ]:
# Area Under Curve (AUC)
# Prediction score of: 0.0, 0.5 and 1.0 are perfect opposite, not better than chance and perfect scores respectively
def train_nn(trainx, trainy, validx, validy):
    params = {
        'objective'          : 'binary',
        'metric'             : 'auc',
        'learning_rate'      : 0.1,
        'max_depth'          : 12,
        'boosting'           : 'gbdt',
        'is_training_metric' : True,
        'seed'               : 42
    }
    trainlgb = lgb.Dataset(trainx, label=trainy)
    validlgb = lgb.Dataset(validx, label=validy)
    return lgb.train(params, trainlgb, num_boost_round=2000, valid_sets=[validlgb], early_stopping_rounds=100, verbose_eval=100)

nn = train_nn(trainx, trainy, validx, validy)
predvy = nn.predict(validx, num_iteration=nn.best_iteration)
predty = nn.predict(trainx, num_iteration=nn.best_iteration)

lgb.plot_importance(nn, figsize=(10,10), height=0.8, grid=False);

In [ ]:
def train_nn(trainx, trainy, validx, validy):
    params = {
        "objective"         : "regression",
        "metric"            : "rmse",
        "num_leaves"        : 30,
        "min_child_samples" : 100,
        "learning_rate"     : 0.1,
        "bagging_fraction"  : 0.7,
        "feature_fraction"  : 0.5,
        "bagging_freq"      : 5,
        "bagging_seed"      : 2018,
        "verbosity"         : -1
    }
    trainlgb = lgb.Dataset(trainx, label=trainy)
    validlgb = lgb.Dataset(validx, label=validy)
    return lgb.train(params, trainlgb, num_boost_round=500, valid_sets=[validlgb], early_stopping_rounds=100, verbose_eval=100)

nn = train_nn(trainx, trainy, validx, validy)
predvy = nn.predict(validx, num_iteration=nn.best_iteration)
predty = nn.predict(trainx, num_iteration=nn.best_iteration)

lgb.plot_importance(nn, figsize=(10,10), height=0.8, grid=False);

In [ ]:
def get_y_predy(labeldf, y, predy):
    return pd.DataFrame({'time':                      labeldf['time'].values,
                         'assetCode':                 labeldf['assetCode'].values,
                         'returnsOpenNextMktres10':   y,
                         'predictedOpenNextMktres10': predy,
                         'allZeros':                  [0] * len(predy),
                         'randomPreds':               (2.0 * np.random.rand(len(predy)) - 1.0) / 10,
                         })

#RMSE (Root Mean Squared Error): 1. Square all Errors (y-predy); 2. take the Mean; 3. then the Root
def rmse(df, col1, col2):
    #return np.sqrt(np.mean((df[col1].values - df[col2].values) ** 2))
    return np.sqrt(metrics.mean_squared_error(df[col1].values, df[col2].values))

#MAE (Mean Absolute Error): 1. abs(y-predy); 2. take the mean
def mae(df, col1, col2):
    #return np.mean(abs(df[col1].values - df[col2].values))
    return metrics.mean_absolute_error(df[col1].values, df[col2].values)

def show_preds(traindf, validdf, name, col):
    print ("trainy - {:s} = RMSE:{:1.4f} (MAE:{:1.4f})".format(name,
        rmse(traindf, 'returnsOpenNextMktres10', col),
        mae( traindf, 'returnsOpenNextMktres10', col)))
    print ("validy - {:s} = RMSE:{:1.4f} (MAE:{:1.4f})".format(name,
        rmse(validdf, 'returnsOpenNextMktres10', col),
        mae( validdf, 'returnsOpenNextMktres10', col)))

trainpreds = get_y_predy(trainx_label, trainy, predty)
validpreds = get_y_predy(validx_label, validy, predvy)
show_preds(trainpreds, validpreds, 'random', 'randomPreds')
show_preds(trainpreds, validpreds, 'all zeros', 'allZeros')
show_preds(trainpreds, validpreds, 'pred validy', 'predictedOpenNextMktres10')
print ("First 10 lines of validation prediction:")
validpreds.head(10)

# Test Data
Two Sigma has written access routines forcing you to iterate through the test data.<br>
Above we initialised: `test_split_df = env.get_prediction_days()`<br>
We iterate one day at a time using: `(market_test_df, news_test_df, prediction_df) = next(test_split_df)`<br>
. for each `prediction_df.assetCode` we set: `prediction_df.confidenceValue` *(those are the only two values)*<br>
. and then apply: `env.predict(prediction_df)`<br>
We finish with: `env.write_submission_file()`<br>

The iterations are split into three chunks in this kernel to retain flexibility whilst testing `test_split_df`:
* First prediction, where I can scrutinise the first day's test data (and define make_random_predictions)
* The remaining predictions
* Last day's test data and prediction can be scrutinised after next() returns 'no more data' error

In [ ]:
def make_predictions(market_test_df, news_test_df, predictions_df):
    market_test_df['time'] = market_test_df['time'].dt.strftime(date_format='%Y-%m-%d').values
    news_test_df['time'] = news_test_df['time'].dt.strftime(date_format='%Y-%m-%d').values
    news_test_df = expand_assetCodes(news_test_df[select_news_cols], select_news_cols)
    market_test_df = split_merge_news(market_test_df, news_test_df, ['time','assetCode'])
    market_test_df.drop(columns=train_unused_cols, inplace=True)
    market_test_df.drop(columns=train_str_cols, inplace=True)
    market_test_df = convert_to_float(market_test_df)
    predictions_df['confidenceValue'] = nn.predict(market_test_df, num_iteration=nn.best_iteration)    
    # RANDOM: predictions_df['confidenceValue'] = 2.0 * np.random.rand(len(predictions_df)) - 1.0
    # ZEROS:  predictions_df['confidenceValue'] = [0] * len(predictions_df)
    return predictions_df

In [ ]:
test_split_df = env.get_prediction_days()
    
print ("day1: first day's test data")
(market_test_df, news_test_df, predictions_template_df) = next(test_split_df)
print ("day1: market shape (rows=assets)", market_test_df.shape)
print ("day1: news info shape (rows=newsinfo)", news_test_df.shape)
print ("day1: pred template shape (cols=assets)", predictions_template_df.shape)
market_start_date = market_test_df['time'].head(1)[0]
news_start_date = news_test_df['time'].head(1)[0]

predictions_df = make_predictions(market_test_df, news_test_df, predictions_template_df)
env.predict(predictions_df)
predications_made = 1
print ("day1: actual predictions shape", predictions_df.shape)

In [ ]:
print ("Doing remaining predictions...")
while True:
    try:
        (market_test_df, news_test_df, predictions_template_df) = next(test_split_df)
        market_end_date = market_test_df['time'].tail(1)[0]
        news_end_date = news_test_df['time'].tail(1)[0]
    except:
        break
    predictions_df = make_predictions(market_test_df, news_test_df, predictions_template_df)
    env.predict(predictions_df)
    predications_made = predications_made + 1
print ("Total",predications_made,"predictions of test made")

In [ ]:
print ("market test dates:",market_start_date,"to",market_end_date)
print ("news test dates:",news_start_date,"to",news_end_date)

# Submission
The file submission.csv will be written to the local directory when you **Commit** your kernel. Check the kernel output outside of the editor and when satisfied press the **Output** tab which will show a **Submit for Competition** button. This is the kernel that will be used for stage two.

In [ ]:
env.write_submission_file()
print([filename for filename in os.listdir('.') if '.csv' in filename])
!head ../working/submission.csv

# Final word regarding the need to restart the kernel
It has been said that the method `make_env` plus iterating through `get_prediction_days` is used to combat cheating, but since the winner will have to make successful predictions about real 2019 data I'm not sure how using any method the programmer desires will give them the upper hand. Surely, if there's a chance that a machine learning programmer can predict future stock prices with great accuracy then why not give them all the help they need?